# Portfolio Workflow

In [1]:
from platform import python_version
import time
from datetime import datetime, timedelta
import os
import pandas as pd
import pandas_datareader as pdr

import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pickle

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, 'tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import trading_factors as alpha_factors
importlib.reload(alpha_factors)
import utils
importlib.reload(utils)
import nonoverlapping_estimator as ai_estimator
importlib.reload(ai_estimator)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')
print(f'Pandas Data Reader version: {pdr.__version__}')

C:\Users\HP-Z440\anaconda3\envs\ml4t\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\HP-Z440\anaconda3\envs\ml4t\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


Sci-Kit version: 0.24.1
Sci-Kit version: 0.24.1
Python version: 3.8.8
Pandas version: 1.4.1
Pandas Data Reader version: 0.10.0


In [2]:
from pathlib import Path

# Make sure we have a data directory
Path('./data').mkdir(parents=True, exist_ok=True) 

# Which account are we interested in
masked_account_number = '#---9216'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories_yahoo.csv'

# Stage 1: Generate Stock Universe

- Gather stocks from specific criteria (SP500 top 50...)
- Use stock sentiment to select stocks
- Gather price histories

## Stock Universe

Here we setup the univers. This needs some work. The long term goal is to use a pipeline process to help select stock that are in the top 500 or something similare.

For now we will use stocks from the portfolio, but stocks of interest (high news items), a list of well known stocks (this also has been augmented with some stocks that made Ameritrade's top 10 movers for a couple of days. This Ameritrade funciton has not been coded yet, but should be add down the line to automate pulling these tickers.

# Price History data

One you have a set of investments you want to work with, you will need to pull some historical data for them.

We will obtain 5 years of price histories. In the end this will provide us with 2 years of factor data since some of the factors are based on 1 year returns.

In [3]:
snp_500_stocks = utils.get_snp500()
stock_universe = utils.reduce_universe_by_sentiment(set(snp_500_stocks.index.to_list()))

Number of stocks in universe: 505


Tickers:   0%|          | 0/505 [00:00<?, ?Finvis Postings/s]

News Tables:   0%|          | 0/503 [00:00<?, ?News Table Items/s]

Mean Sentiment: 2.0712732959664213 with a standared deviation of: 1.688777463936004 providing a cutoff of: 0.38249583203041726
New number of stocks in universe: 446


In [4]:
from dateutil.relativedelta import relativedelta
number_of_years = 5
start = datetime.today() - relativedelta(years = number_of_years)
yahoo = pdr.yahoo.daily.YahooDailyReader(symbols=stock_universe, start=start, adjust_price=True, interval='d', get_actions=False, adjust_dividends=True, pause=0.2)
price_histories = yahoo.read()
yahoo.close()
price_histories.to_csv(price_histories_file_name, index=True)

In [5]:
price_histories

Attributes Adj_Ratio                                                        \
Symbols            A       AAL       AAP     AAPL      ABBV       ABC ABMD   
Date                                                                         
2017-03-27  0.960623  0.968483  0.963511  0.94416  0.794469  0.917661  1.0   
2017-03-28  0.960623  0.968483  0.963511  0.94416  0.794469  0.917661  1.0   
2017-03-29  0.960623  0.968483  0.963511  0.94416  0.794469  0.917661  1.0   
2017-03-30  0.960623  0.968482  0.963510  0.94416  0.794469  0.917661  1.0   
2017-03-31  0.963007  0.968483  0.963510  0.94416  0.794469  0.917661  1.0   
...              ...       ...       ...      ...       ...       ...  ...   
2022-03-18  1.000000  1.000000  1.000000  1.00000  1.000000  1.000000  1.0   
2022-03-21  1.000000  1.000000  1.000000  1.00000  1.000000  1.000000  1.0   
2022-03-22  1.000000  1.000000  1.000000  1.00000  1.000000  1.000000  1.0   
2022-03-23  1.000000  1.000000  1.000000  1.00000  1.000000  1.000000  1.0   
2022-03-24  1.000000  1.000000  1.000000  1.00000  1.000000  1.000000  1.0   

Attributes                           ...     Volume                         \
Symbols          ACN ADBE       ADI  ...       WYNN        XEL         XOM   
Date                                 ...                                     
2017-03-27  0.922028  1.0  0.906076  ...  3487200.0  2433800.0   8481200.0   
2017-03-28  0.922028  1.0  0.906076  ...  2148000.0  4144500.0  11028600.0   
2017-03-29  0.922028  1.0  0.906076  ...  1213300.0  2646700.0   8495500.0   
2017-03-30  0.922028  1.0  0.906076  ...  1685900.0  2131200.0  44704900.0   
2017-03-31  0.922028  1.0  0.906076  ...  1979600.0  2384700.0  21750400.0   
...              ...  ...       ...  ...        ...        ...         ...   
2022-03-18  1.000000  1.0  1.000000  ...  3712800.0  5743500.0  41445900.0   
2022-03-21  1.000000  1.0  1.000000  ...  2647900.0  2559000.0  33089300.0   
2022-03-22  1.000000  1.0  1.000000  ...  2732300.0  2351000.0  23132200.0   
2022-03-23  1.000000  1.0  1.000000  ...  1636700.0  2511300.0  27716300.0   
2022-03-24  1.000000  1.0  1.000000  ...  2444607.0  2208072.0  17793299.0   

Attributes                                                                   \
Symbols          XRAY        XYL        YUM        ZBH      ZBRA       ZION   
Date                                                                          
2017-03-27   953500.0   878900.0  4936600.0  1493603.0  217900.0  3237500.0   
2017-03-28   930200.0   995200.0  3824100.0   915567.0  284900.0  2392400.0   
2017-03-29  1103100.0  1046700.0  2714600.0   522622.0  257700.0  2055900.0   
2017-03-30   621000.0   969200.0  3254100.0   542192.0  312100.0  3212100.0   
2017-03-31  1061400.0  1230100.0  2410400.0   886727.0  313800.0  2575200.0   
...               ...        ...        ...        ...       ...        ...   
2022-03-18  6863700.0  4327800.0  3361800.0  2948400.0  930700.0  4135800.0   
2022-03-21  2982200.0  1643700.0  1930700.0  1444100.0  491300.0  1779800.0   
2022-03-22  2983300.0  1678300.0  1179300.0  1819700.0  455800.0  1805000.0   
2022-03-23  1584900.0  1819800.0  1798900.0  2420000.0  325800.0  1498600.0   
2022-03-24  1328067.0   880990.0  1554220.0  2204310.0  307258.0   934172.0   

Attributes             
Symbols           ZTS  
Date                   
2017-03-27  3168400.0  
2017-03-28  2328200.0  
2017-03-29  2269900.0  
2017-03-30  2697500.0  
2017-03-31  3163800.0  
...               ...  
2022-03-18  2613700.0  
2022-03-21  1891000.0  
2022-03-22  2374700.0  
2022-03-23  2115200.0  
2022-03-24  3445817.0  

[1259 rows x 2676 columns]

In [6]:
test_read = pd.read_csv(price_histories_file_name, header=[0, 1], index_col=[0], low_memory=False)

In [7]:

test_read

Attributes Adj_Ratio                                                        \
Symbols            A       AAL       AAP     AAPL      ABBV       ABC ABMD   
Date                                                                         
2017-03-27  0.960623  0.968483  0.963511  0.94416  0.794469  0.917661  1.0   
2017-03-28  0.960623  0.968483  0.963511  0.94416  0.794469  0.917661  1.0   
2017-03-29  0.960623  0.968483  0.963511  0.94416  0.794469  0.917661  1.0   
2017-03-30  0.960623  0.968482  0.963510  0.94416  0.794469  0.917661  1.0   
2017-03-31  0.963007  0.968483  0.963510  0.94416  0.794469  0.917661  1.0   
...              ...       ...       ...      ...       ...       ...  ...   
2022-03-18  1.000000  1.000000  1.000000  1.00000  1.000000  1.000000  1.0   
2022-03-21  1.000000  1.000000  1.000000  1.00000  1.000000  1.000000  1.0   
2022-03-22  1.000000  1.000000  1.000000  1.00000  1.000000  1.000000  1.0   
2022-03-23  1.000000  1.000000  1.000000  1.00000  1.000000  1.000000  1.0   
2022-03-24  1.000000  1.000000  1.000000  1.00000  1.000000  1.000000  1.0   

Attributes                           ...     Volume                         \
Symbols          ACN ADBE       ADI  ...       WYNN        XEL         XOM   
Date                                 ...                                     
2017-03-27  0.922028  1.0  0.906076  ...  3487200.0  2433800.0   8481200.0   
2017-03-28  0.922028  1.0  0.906076  ...  2148000.0  4144500.0  11028600.0   
2017-03-29  0.922028  1.0  0.906076  ...  1213300.0  2646700.0   8495500.0   
2017-03-30  0.922028  1.0  0.906076  ...  1685900.0  2131200.0  44704900.0   
2017-03-31  0.922028  1.0  0.906076  ...  1979600.0  2384700.0  21750400.0   
...              ...  ...       ...  ...        ...        ...         ...   
2022-03-18  1.000000  1.0  1.000000  ...  3712800.0  5743500.0  41445900.0   
2022-03-21  1.000000  1.0  1.000000  ...  2647900.0  2559000.0  33089300.0   
2022-03-22  1.000000  1.0  1.000000  ...  2732300.0  2351000.0  23132200.0   
2022-03-23  1.000000  1.0  1.000000  ...  1636700.0  2511300.0  27716300.0   
2022-03-24  1.000000  1.0  1.000000  ...  2444607.0  2208072.0  17793299.0   

Attributes                                                                   \
Symbols          XRAY        XYL        YUM        ZBH      ZBRA       ZION   
Date                                                                          
2017-03-27   953500.0   878900.0  4936600.0  1493603.0  217900.0  3237500.0   
2017-03-28   930200.0   995200.0  3824100.0   915567.0  284900.0  2392400.0   
2017-03-29  1103100.0  1046700.0  2714600.0   522622.0  257700.0  2055900.0   
2017-03-30   621000.0   969200.0  3254100.0   542192.0  312100.0  3212100.0   
2017-03-31  1061400.0  1230100.0  2410400.0   886727.0  313800.0  2575200.0   
...               ...        ...        ...        ...       ...        ...   
2022-03-18  6863700.0  4327800.0  3361800.0  2948400.0  930700.0  4135800.0   
2022-03-21  2982200.0  1643700.0  1930700.0  1444100.0  491300.0  1779800.0   
2022-03-22  2983300.0  1678300.0  1179300.0  1819700.0  455800.0  1805000.0   
2022-03-23  1584900.0  1819800.0  1798900.0  2420000.0  325800.0  1498600.0   
2022-03-24  1328067.0   880990.0  1554220.0  2204310.0  307258.0   934172.0   

Attributes             
Symbols           ZTS  
Date                   
2017-03-27  3168400.0  
2017-03-28  2328200.0  
2017-03-29  2269900.0  
2017-03-30  2697500.0  
2017-03-31  3163800.0  
...               ...  
2022-03-18  2613700.0  
2022-03-21  1891000.0  
2022-03-22  2374700.0  
2022-03-23  2115200.0  
2022-03-24  3445817.0  

[1259 rows x 2676 columns]